In [1]:
import datetime
import time
import base64
import hmac
import hashlib
import requests
import json
import pandas as pd
import os
from dotenv import load_dotenv

%matplotlib inline

# silence warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
dotenv_path = 'C:/Desktop/my_gitrepos/project-2/ndax_data/n.env'
load_dotenv(dotenv_path)

True

In [3]:
api_key = os.getenv('ndax_api_key')
api_secret = os.getenv('ndax_api_secret')
user_id = os.getenv('user_id')

In [4]:
def get_user_accounts(user_id):
    endpoint = "https://api.ndax.io/ndaxapi/ep/v1/account/GetUserAccounts"
    params = {
        'userId': user_id
    }
    headers = {
        'X-NDAX-KEY': api_key,
        'X-NDAX-PAYLOAD': '',
        'X-NDAX-SIGNATURE': ''
    }
    response = requests.get(endpoint, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        account_ids = data.split(",")
        df = pd.DataFrame({"AccountID": account_ids})
        return df
    else:
        print("Request failed with status code:", response.status_code)
        return None
        
# Call the function and store the DataFrame
user_accounts_df = get_user_accounts(user_id)
user_accounts_df

Request failed with status code: 404


In [9]:
def get_ndax_tickers():
    endpoint = "https://api.ndax.io/ndaxapi/ep/v1/ticker"
    response = requests.get(endpoint)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)  # Creating a DataFrame from the ticker data
        return df
    else:
        print("Request failed with status code:", response.status_code)
        return None
# Call the function and store the DataFrame
ndax_tickers_df = get_ndax_tickers()
ndax_tickers_df

BTC_CAD  BCH_CAD        ETH_CAD      XRP_CAD      LTC_CAD   
base_id       1.000000e+00   1831.0    1027.000000      52.0000      2.00000  \
quote_id      0.000000e+00      0.0       0.000000       0.0000      0.00000   
last_price    3.603724e+04      1.0    2463.380000       0.6157    117.22000   
base_volume   3.836824e+01      0.0     175.372700  170882.0000    240.74300   
quote_volume  1.378363e+06      0.0  430588.181397  105288.5845  27971.96662   

              BTC_USD    EOS_CAD      XLM_CAD       DOGE_CAD       ADA_CAD   
base_id          1.00  1765.0000     512.0000      74.000000   2010.000000  \
quote_id         0.00     0.0000       0.0000       0.000000      0.000000   
last_price    6547.67     1.1510       0.1182       0.096001      0.494350   
base_volume      0.00    23.5000  135697.0000  200391.000000  69018.200000   
quote_volume     0.00    26.7118   16022.6523   19251.335005  33942.052279   

              ...   NEAR_CAD  CELO_CAD       VET_CAD      GALA_CAD  RUNE_CAD   
base_id       ...  6535.0000  5567.000   3077.000000    7080.00000  9905.000  \
quote_id      ...     0.0000     0.000      0.000000       0.00000     0.000   
last_price    ...     2.1700     0.670      0.026500       0.04098     1.354   
base_volume   ...   238.0000   368.100  31279.000000  331504.00000     0.000   
quote_volume  ...   511.3664   248.735    826.619672   13153.46923     0.000   

              USDT_USD  USDC_USD  XRP_USD      ARB_CAD    ATOM_CAD  
base_id       825.0000    3408.0       52  21129.00000  3794.00000  
quote_id        0.0000       0.0        0      0.00000     0.00000  
last_price      1.0101       1.0        0      1.49680    14.21300  
base_volume     0.0000       0.0        0   4092.00000    20.95000  
quote_volume    0.0000       0.0        0   6063.51488   297.93208  

[5 rows x 43 columns]

In [10]:
def get_recent_trades(market_pair):
    endpoint = f"https://api.ndax.io/ndaxapi/ep/v1/trades/{market_pair}"

    response = requests.get(endpoint)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        return df
    else:
        print("Request failed with status code:", response.status_code)
        return None

# Set the parameter for the Trades request
market_pair = "ETHCAD"  # Replace with the desired market pair, e.g., "BTCCAD", "ETHCAD"

# Call the function and store the DataFrame
recent_trades_df = get_recent_trades(market_pair)
recent_trades_df

Request failed with status code: 404


In [11]:
def get_summary():
    endpoint = "https://api.ndax.io/ndaxapi/ep/v1/summary"
    response = requests.get(endpoint)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        return df
    else:
        print("Request failed with status code:", response.status_code)
        return None

# Call the function and store the DataFrame
summary_df = get_summary()
summary_df

trading_pairs    last_price    lowest_ask   highest_bid   base_volume   
0        BTC_CAD  36037.240000  36038.050000  35905.000000  3.836824e+01  \
1        BCH_CAD      1.000000      0.000000      0.000000  0.000000e+00   
2        ETH_CAD   2463.940000   2463.910000   2450.010000  1.757785e+02   
3        XRP_CAD      0.615700      0.616900      0.613100  1.708820e+05   
4        LTC_CAD    117.220000    116.390000    115.440000  2.407430e+02   
5        BTC_USD   6547.670000      0.000000      0.000000  0.000000e+00   
6        EOS_CAD      1.151000      1.153000      1.130000  2.350000e+01   
7        XLM_CAD      0.118200      0.118200      0.118000  1.356970e+05   
8       DOGE_CAD      0.096001      0.096463      0.096001  2.003910e+05   
9        ADA_CAD      0.494350      0.497790      0.488350  6.901820e+04   
10       LTC_USD      0.000000      0.000000      0.000000  0.000000e+00   
11      USDT_CAD      1.339600      1.365100      1.343200  5.249700e+03   
12      LINK_CAD      8.520100      8.635200      8.500000  3.098200e+02   
13      BTC_USDT  29869.010000  26761.060000  25842.780000  0.000000e+00   
14       DOT_CAD      7.160600      7.235500      7.157700  2.125980e+03   
15       UNI_CAD      6.640000      6.800000      6.640000  2.418600e+02   
16       GRT_CAD      0.157600      0.159000      0.156300  2.999200e+04   
17      COMP_CAD     46.650000     47.670000     46.510000  5.300000e-01   
18      AAVE_CAD    377.779300      0.000000      0.000000  0.000000e+00   
19      AAVE_CAD     85.938000     86.907000     85.463000  1.737200e+01   
20     MATIC_CAD      1.209950      1.209940      1.202680  1.627970e+04   
21      USDC_CAD      1.365200      1.367500      1.365200  3.616830e+05   
22      BTC_USDC  20076.700000      0.000000      0.000000  0.000000e+00   
23      SHIB_CAD      0.000012      0.000012      0.000012  2.470391e+09   
24       FTM_CAD      0.452700      0.453800      0.448300  2.232720e+05   
25      LUNC_CAD      0.000114      0.000117      0.000112  6.884488e+05   
26       AXS_CAD      9.370000      9.360000      9.180000  1.013100e+02   
27      MANA_CAD      0.628300      0.636600      0.624700  1.180000e+03   
28      HBAR_CAD      0.070000      0.070100      0.068700  9.698500e+04   
29      AVAX_CAD     19.470000     19.290000     19.030000  2.047700e+02   
30       SOL_CAD     26.480000     26.550000     26.480000  8.485100e+02   
31      ALGO_CAD      0.207200      0.207200      0.203900  3.438900e+04   
32      SAND_CAD      0.698800      0.692400      0.679300  3.781000e+03   
33       TLM_CAD      0.021300      0.021700      0.021200  1.215900e+04   
34      NEAR_CAD      2.170000      2.156000      2.113000  2.380000e+02   
35      CELO_CAD      0.670000      0.686000      0.674000  3.681000e+02   
36       VET_CAD      0.026500      0.026430      0.026050  3.127900e+04   
37      GALA_CAD      0.040980      0.041260      0.040710  3.315040e+05   
38      RUNE_CAD      1.354000      0.000000      0.000000  0.000000e+00   
39      USDT_USD      1.010100      0.000000      0.000000  0.000000e+00   
40      USDC_USD      1.000000      0.000000      0.000000  0.000000e+00   
41       XRP_USD      0.000000      0.000000      0.000000  0.000000e+00   
42       ARB_CAD      1.496800      1.511100      1.489500  4.092000e+03   
43      ATOM_CAD     14.213000     14.391000     14.216000  2.095000e+01   

    quote_volume  price_change_percent_24h  highest_price_24h   
0   1.378363e+06                 -0.040802       36258.510000  \
1   0.000000e+00                  0.000000           0.000000   
2   4.315880e+05                  0.921587        2475.830000   
3   1.052886e+05                  0.440457           0.622400   
4   2.797197e+04                 -0.593623         117.220000   
5   0.000000e+00                  0.000000           0.000000   
6   2.671180e+01                  1.768347           1.151000   
7   1.602265e+04                  0.339559           0.119400 

In [15]:
def get_summary():
    endpoint = "https://api.ndax.io/ndaxapi/ep/v1/summary"
    response = requests.get(endpoint)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        df['Timestamp'] = int(time.time())  # Add a timestamp column with the current time
        return df
    else:
        print("Request failed with status code:", response.status_code)
        return None

# Call the function and store the DataFrame
summary_df = get_summary()
summary_df.columns

Index(['trading_pairs', 'last_price', 'lowest_ask', 'highest_bid',
       'base_volume', 'quote_volume', 'price_change_percent_24h',
       'highest_price_24h', 'lowest_price_24h', 'Timestamp'],
      dtype='object')

In [26]:
def get_account_info(account_id):
    endpoint = "https://api.ndax.io/ndaxapi/ep/v1/account/{account_id}/info"
    response = requests.get(endpoint)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data, index=[0])
        return df
    else:
        print("Request failed with status code:", response.status_code)
        return None

# Call the function and store the DataFrame
#account_info_df = get_account_info(account_id)
#account_info_df

In [ ]:
# Set the parameters for the GetUserAccounts request
oms_id = 0  # The Order Management System ID
user_id = 0  # The ID of the user
user_name = ""  # The name of the user

def get_user_accounts(oms_id, user_id, user_name):
    endpoint = "https://api.ndax.io/ndaxapi/ep/v1/user/accounts"
    request_data = {
        "omsId": oms_id,
        "userId": user_id,
        "userName": user_name
    }

    response = requests.post(endpoint, json=request_data)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        return df
    else:
        print("Request failed with status code:", response.status_code)
        return None

# Call the function and store the DataFrame
user_accounts_df = get_user_accounts(oms_id, user_id, user_name)
user_accounts_df

In [17]:
def get_recent_trades(market_pair):
    endpoint = f"https://api.ndax.io/ndaxapi/ep/v1/trades/{market_pair}"
    response = requests.get(endpoint)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        return df
    else:
        print("Request failed with status code:", response.status_code)
        return None

# Set the parameter for the Trades request
market_pair = "BTCCAD"  # Replace with the desired market pair, e.g., "BTCCAD", "ETHCAD"

# Call the function and store the DataFrame
recent_trades_df = get_recent_trades(market_pair)
recent_trades_df

Request failed with status code: 404


In [22]:
def get_trades(market_pair):
    endpoint = f"https://api.ndax.io/ndaxapi/ep/v1/trades/{market_pair}"
    response = requests.get(endpoint)

    if response.status_code == 200:
        data = response.json()
        trades = data['Trades']

        # Extracting the required fields from each trade
        trade_data = [{'Date': trade['DateTime'],
                       'Open': trade['Price'],
                       'High': trade['Price'],
                       'Low': trade['Price'],
                       'Close': trade['Price'],
                       'Volume': trade['Quantity']} for trade in trades]

        df = pd.DataFrame(trade_data)
        return df
    else:
        print("Request failed with status code:", response.status_code)
        return None

# Set the parameter for the Trades request
market_pair = "ETHCAD"  # Replace with the desired market pair, e.g., "BTCCAD", "ETHCAD"

# Call the function and store the DataFrame
recent_trades_df = get_trades(market_pair)
recent_trades_df

Request failed with status code: 404


In [23]:
def get_historical_trades(market_pair, start_time, end_time):
    endpoint = "https://api.ndax.io/ndaxapi/ep/v1/history/Trades/{market_pair}"
    params = {
        'start': start_time,
        'end': end_time
    }
    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        data = response.json()
        trades = data['Trades']

        # Extracting the required fields from each trade
        trade_data = [{'Date': trade['DateTime'],
                       'Open': trade['Price'],
                       'High': trade['Price'],
                       'Low': trade['Price'],
                       'Close': trade['Price'],
                       'Volume': trade['Quantity']} for trade in trades]

        df = pd.DataFrame(trade_data)
        return df
    else:
        print("Request failed with status code:", response.status_code)
        return None        

In [25]:
market_pair = 'ETHCAD'
start_time = 1620000000  # Replace with the desired start time in Unix timestamp format
end_time = 1630000000  # Replace with the desired end time in Unix timestamp format

historical_trades_df = get_historical_trades(market_pair, start_time, end_time)
historical_trades_df

Request failed with status code: 404


In [41]:
def get_summary():
    endpoint = "https://api.ndax.io/ndaxapi/ep/v1/summary"
    response = requests.get(endpoint)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        return df
    else:
        print("Request failed with status code:", response.status_code)
        return None

# Call the function and store the DataFrame
summary_df = get_summary()
summary_df.columns

Index(['trading_pairs', 'last_price', 'lowest_ask', 'highest_bid',
       'base_volume', 'quote_volume', 'price_change_percent_24h',
       'highest_price_24h', 'lowest_price_24h'],
      dtype='object')

In [42]:
def get_recent_trades(market_pair):
    endpoint = f"https://api.ndax.io/ndaxapi/ep/v1/trades/{market_pair}"
    response = requests.get(endpoint)

    if response.status_code == 200:
        data = response.json()
        trades = data['Trades']

        trade_data = [{'Timestamp': trade['DateTime'],
                       'Open': trade['Price'],
                       'High': trade['Price'],
                       'Low': trade['Price'],
                       'Close': trade['Price'],
                       'Volume': trade['Quantity']} for trade in trades]

        df = pd.DataFrame(trade_data)
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')  # Convert Timestamp to datetime format
        df.set_index('Timestamp', inplace=True)  # Set Timestamp as the index
        return df
    else:
        print("Request failed with status code:", response.status_code)
        return None

# Set the parameter for the Trades request
market_pair = "ETHCAD"  # Replace with the desired market pair, e.g., "BTCCAD", "ETHCAD"

# Call the function and store the DataFrame
recent_trades_df = get_recent_trades(market_pair)
recent_trades_df

Request failed with status code: 404
